In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit - Sequential Agents

### Install adk

In [1]:
%pip install --upgrade --quiet google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.8 MB/s eta 0:00:00


In [2]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[2]
print(MODEL_NAME)

gemini-2.5-flash-preview-04-17


In [3]:
# @title Authentication to access to GCP
import sys
from pprint import pprint

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Initialize Vertex AI

* Please refer to configuration to **initialize Vertex AI : [Link](https://google.github.io/adk-docs/get-started/quickstart/#set-up-the-model)**
* You can set up api key rather than usning Vertex AI.


In [4]:
import os

GOOGLE_GENAI_USE_VERTEXAI = "TRUE"                            #@param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = GOOGLE_GENAI_USE_VERTEXAI

GOOGLE_CLOUD_PROJECT = "ai-hangsik"                           #@param {type:"string"}
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT

GOOGLE_CLOUD_LOCATION = "us-central1"                         #@param {type:"string"}
os.environ["GOOGLE_CLOUD_LOCATION"] = GOOGLE_CLOUD_LOCATION




## adk with tools

In [7]:
import asyncio

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.adk.tools import google_search
from google.adk.agents import SequentialAgent
from google.adk.agents import ParallelAgent


In [26]:
positive_agent = LlmAgent(
    name="PositiveAgent",
    model=MODEL_NAME,
    instruction="""You are a helpful AI assistant to find positive side of given technology
                   Add the title "## Positive aspects" to the response.

    """,
    description="Find positive aspects for the given technology",
    output_key="positive_perspectives",
    tools=[google_search],
)

negative_agent = LlmAgent(
    name="NegativeAgent",
    model=MODEL_NAME,
    instruction="""You are a helpful AI assistant to find negative side of given technology
                       Add the title "## Negative aspects" to the response.

    """,
    description="Find negative aspects for the given technology",
    output_key="negative_perspectives",
    tools=[google_search],

)

conculusion_agent = LlmAgent(
    name="ConclusionAgent",
    model=MODEL_NAME,
    instruction="""You are a helpful AI assistant to draw conclusion by reasoning the both side of positive and negative
                   Add the title "## Conclusion" to the response.


    """,
    description="Draw conculusion.",
    output_key="conculusion",
    tools=[google_search],
)

review_agent = LlmAgent(
    name="ReviewAgent",
    model=MODEL_NAME,
    instruction="""You are a helpful AI assistant to review the conclusion
                   Add the title "## Review Conclusion" to the response.

    """,
    description="Review conculusion.",
    output_key="review",
    tools=[google_search],
)

In [27]:
parallel_research_agent = ParallelAgent(
    name="ParallelResearchAgent",
    sub_agents=[positive_agent, negative_agent],
    description="Runs multiple research agents in parallel to gather information."
)

In [28]:

pipeline_agent = SequentialAgent(
    name="PipelineAgent",
    sub_agents=[parallel_research_agent, conculusion_agent, review_agent],
    description="Executes a sequence of positive_agent, negative_agent, and refactoring.",
)


In [29]:
app_name = "Assistant"
user_id = "shins"

query = "what is Generative AI?"

session_service = InMemorySessionService()
session = session_service.create_session(app_name=app_name, user_id=user_id)

runner = Runner(agent=pipeline_agent,
                app_name=session.app_name,
                session_service=session_service)

content = types.Content(role='user', parts=[types.Part(text=query)])

events = runner.run(user_id=session.user_id,
                    session_id=session.id,
                    new_message=content)

for event in events:
    print(event.content.parts[0].text)

    # if event.is_final_response():
    #     final_response = event.content.parts[0].text
    #     print(f"Agent Response: {final_response}\n")
    # else:
    #     print("No results")

Generative Artificial Intelligence (AI), often shortened to Generative AI or GenAI, is a category of artificial intelligence that focuses on creating new content and ideas [2, 7, 9]. Unlike traditional AI that might analyze or classify existing data, generative models learn the patterns and structures within vast datasets and use this knowledge to produce novel outputs [3, 4, 6]. These outputs can take many forms, including text, images, audio, code, simulations, and videos [2, 4]. Essentially, Generative AI learns how the training data is put together and then generates new data that is similar but not identical to the original [3, 9].

## Positive aspects

Generative AI offers numerous benefits across various fields:

*   **Enhanced Creativity and Innovation:** It can generate new ideas, designs, and content, augmenting human creativity [1, 5, 9]. This includes brainstorming product ideas, creating unique images or music, and exploring topics from different viewpoints [5, 10].
*   **

## End of notebook